In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("./Datasets/casos_covid19.csv")
df = df[df["clasificacion"]=="confirmado"]

C:\Users\ignac\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10,11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Cases by date
### Aggregating the number of cases/deaths by date 
### We first need to make the date into a readable format and drop the data points without a date

In [2]:
def transform_date(date):
    decoder = {"JAN":"01","FEB" : "02","MAR" : "03", "APR" : "04", "MAY" : "05", "JUN" : "06", "JUL" : "07","AUG" : "08","SEP" : "09","OCT" : "10","NOV" : "11","DEC" : "12"}
    return date[:2] + "-" + decoder[date[2:5]] + "-" + date[5:9]

df = df.dropna(subset=["fecha_apertura_snvs"])
df["fecha_apertura_snvs"] = df["fecha_apertura_snvs"].apply(transform_date)
df["fecha_apertura_snvs"] = pd.to_datetime(df["fecha_apertura_snvs"], format="%d-%m-%Y")
df.sort_values(by="fecha_apertura_snvs")

,numero_de_caso,fecha_apertura_snvs,fecha_toma_muestra,fecha_clasificacion,provincia,barrio,comuna,genero,edad,clasificacion,fecha_fallecimiento,fallecido,fecha_alta,tipo_contagio
68625,734792,2020-03-02,01MAR2020:00:00:00.000000,11MAR2020:00:00:00.000000,CABA,PUERTO MADERO,1.0,masculino,44.0,confirmado,NaN,NaN,13MAR2020:00:00:00.000000,Importado
1124995,735857,2020-03-03,15OCT2020:00:00:00.000000,17OCT2020:00:00:00.000000,CABA,PALERMO,14.0,masculino,50.0,confirmado,NaN,NaN,NaN,Importado
38430,738487,2020-03-05,05MAR2020:00:00:00.000000,07MAR2020:00:00:00.000000,CABA,SAN TELMO,1.0,masculino,66.0,confirmado,07MAR2020:00:00:00.000000,si,NaN,Importado
68623,739066,2020-03-05,05MAR2020:00:00:00.000000,06MAR2020:00:00:00.000000,CABA,FLORES,7.0,femenino,73.0,confirmado,NaN,NaN,14MAR2020:00:00:00.000000,Importado
68626,739450,2020-03-05,05MAR2020:00:00:00.000000,31MAR2020:00:00:00.000000,CABA,BELGRANO,13.0,masculino,25.0,confirmado,NaN,NaN,13MAR2020:00:00:00.000000,Importado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196225,18757175,2021-08-15,14AUG2021:00:00:00.000000,14AUG2021:00:00:00.000000,Buenos Aires,NaN,NaN,femenino,46.0,confirmado,NaN,NaN,NaN,En Investigación
196224,18756679,2021-08-15,13AUG2021:00:00:00.000000,13AUG2021:00:00:00.000000,CABA,VILLA GRAL. MITRE,11.0,masculino,42.0,confirmado,NaN,NaN,NaN,En Investigación
196223,18756662,2021-08-15,13AUG2021:00:00:00.000000,13AUG2021:00:00:00.000000,Buenos Aires,NaN,NaN,masculino,49.0,confirmado,NaN,NaN,NaN,En Investigación
343733,18761785,2021-08-15,NaN,15AUG2021:00:00:00.000000,CABA,SAN NICOLAS,1.0,masculino,32.0,confirmado,NaN,NaN,NaN,En Investigación


In [3]:
cases_by_date = df.groupby("fecha_apertura_snvs", as_index=False).count()
cases_by_date = cases_by_date[["fecha_apertura_snvs","numero_de_caso"]] # We keep only the dates and number of cases 
cases_by_date = cases_by_date.rename(columns={"fecha_apertura_snvs":"date", "numero_de_caso":"cases"})

In [4]:
deaths_by_date = df[df["fallecido"]=="si"].groupby("fecha_apertura_snvs", as_index=False).count()
deaths_by_date = deaths_by_date[["fecha_apertura_snvs","numero_de_caso"]] # We keep only the dates and number of cases 
deaths_by_date = deaths_by_date.rename(columns={"fecha_apertura_snvs":"date", "numero_de_caso":"deaths"}) # We rename those two columns to date and deaths

In [5]:
data_by_date = pd.merge(cases_by_date,deaths_by_date,on="date")

In [6]:
cases_by_date["casesMA"] = cases_by_date["cases"].rolling(window=7).mean()
deaths_by_date["deathsMA"] = deaths_by_date["deaths"].rolling(window=7).mean()

In [7]:
cases_by_date.to_csv("./Datasets/cases_by_date.csv")
deaths_by_date.to_csv("./Datasets/deaths_by_date.csv")

# Cases by neighborhoods
### We use GeoPandas to create a dataset with geographical information about each neighborhood with the data corresponding to population and cases/deaths 

In [8]:
import geopandas as gpd
neighborhoods = gpd.read_file("./Datasets/barrios.geojson")       # Neighborhoods geographic data
census = gpd.read_file("./Datasets/caba_radios_censales.geojson") # Census data

In [11]:
population_by_neighborhood = census[["BARRIO","POBLACION"]].groupby("BARRIO").sum()
neighborhoods = neighborhoods.join(on="barrio", other=population_by_neighborhood) # Aggregate data of population by neighborhood

neighborhoods = neighborhoods.join(on="barrio",other=df.dropna(subset=["barrio"]).groupby("barrio").count()["numero_de_caso"]) # Aggregate data of cases by neighborhood
neighborhoods = neighborhoods.join(on="barrio",other=df.dropna(subset=["barrio","fallecido"]).groupby("barrio").count()["fallecido"]) # Aggregate data of deaths by neighborhood

neighborhoods["case_density"] = 1000* neighborhoods["numero_de_caso"]/neighborhoods["POBLACION"] # Density per 1000 habitants
neighborhoods["death_density"] = 1000* neighborhoods["fallecido"]/neighborhoods["POBLACION"]     #

neighborhoods = neighborhoods.rename(columns={"numero_de_caso":"cases", "POBLACION":"population", "fallecido":"deaths"}) # Rename the relevant columns for clarity

In [12]:
neighborhoods.head()

,barrio,comuna,perimetro,area,geometry,population,cases,deaths,case_density,death_density,population,cases,deaths
0,CHACARITA,15,7724.852955,3.115707e+06,"POLYGON ((-58.45282 -34.59599, -58.45366 -34.5...",27761.0,5369,112,193.400814,4.034437,27761.0,5369,112
1,PATERNAL,15,7087.513295,2.229829e+06,"POLYGON ((-58.46558 -34.59656, -58.46562 -34.5...",19717.0,3489,87,176.953898,4.412436,19717.0,3489,87
2,VILLA CRESPO,15,8131.857075,3.615978e+06,"POLYGON ((-58.42375 -34.59783, -58.42495 -34.5...",81959.0,13469,286,164.338267,3.489550,81959.0,13469,286
3,VILLA DEL PARQUE,11,7705.389797,3.399596e+06,"POLYGON ((-58.49461 -34.61487, -58.49479 -34.6...",55273.0,7692,176,139.163787,3.184195,55273.0,7692,176
4,ALMAGRO,5,8537.901368,4.050752e+06,"POLYGON ((-58.41287 -34.61412, -58.41282 -34.6...",131699.0,21480,498,163.099188,3.781350,131699.0,21480,498


In [13]:
neighborhoods.to_file("./Datasets/covid_data_by_neighborhood.geojson", driver='GeoJSON')

# Cases by month on each neighborhood
### We put everything together to get a dataset with the number of cases/deaths by month on each neighborhood
### We use the pivot_table function from pandas to do so.

In [14]:
df["month-year"] = df["fecha_apertura_snvs"].apply(lambda x: x.month_name()[:3]+str(x.year))
df["cases"] = df["numero_de_caso"].apply(lambda x:1)
monthly_data = df.pivot_table(index="barrio", columns ="month-year", values = "cases", aggfunc=np.sum)

In [15]:
monthly_data = monthly_data.reset_index()

In [16]:
monthly_data.head()

month-year,barrio,Apr2020,Apr2021,Aug2020,Aug2021,Dec2020,Feb2021,Jan2021,Jul2020,Jul2021,Jun2020,Jun2021,Mar2020,Mar2021,May2020,May2021,Nov2020,Oct2020,Sep2020
0,AGRONOMIA,4.0,285.0,100.0,55.0,57.0,91.0,156.0,70.0,95.0,28.0,167.0,3.0,114.0,5.0,252.0,38.0,72.0,115.0
1,ALMAGRO,45.0,3373.0,1607.0,306.0,727.0,1009.0,1578.0,1391.0,1142.0,716.0,1774.0,17.0,1364.0,179.0,3261.0,548.0,987.0,1456.0
2,BALVANERA,54.0,3955.0,2297.0,319.0,757.0,938.0,1590.0,2271.0,1156.0,1326.0,2090.0,42.0,1549.0,328.0,3954.0,575.0,1061.0,1784.0
3,BARRACAS,23.0,2674.0,1617.0,148.0,491.0,479.0,824.0,1825.0,653.0,1883.0,1380.0,3.0,858.0,470.0,2605.0,275.0,504.0,992.0
4,BELGRANO,60.0,3008.0,1065.0,418.0,875.0,1011.0,2089.0,852.0,1057.0,274.0,1568.0,30.0,1463.0,110.0,2582.0,477.0,848.0,1127.0


In [18]:
# Ordered months for column names
ordered_months = ["Mar2020", 
                   "Apr2020", 
                   "May2020", 
                   "Jun2020", 
                   "Jul2020",
                   "Aug2020",
                   "Sep2020",
                   "Oct2020",
                   "Nov2020",
                   "Dec2020",
                   "Jan2021",
                   "Feb2021",
                   "Mar2021", 
                   "Apr2021", 
                   "May2021", 
                   "Jun2021", 
                   "Jul2021"]

In [19]:
cases_by_month = neighborhoods.merge(monthly_data)[["barrio","geometry","population"]+ordered_months]
cases_by_month = cases_by_month.fillna(0)

In [23]:
cases_by_month.to_file("./Datasets/cases_by_month.geojson", driver='GeoJSON')

In [21]:
cases_by_month = neighborhoods.merge(monthly_data)

In [22]:
cases_by_month.head()

,barrio,comuna,perimetro,area,geometry,population,cases,deaths,case_density,death_density,...,Jul2021,Jun2020,Jun2021,Mar2020,Mar2021,May2020,May2021,Nov2020,Oct2020,Sep2020
0,CHACARITA,15,7724.852955,3.115707e+06,"POLYGON ((-58.45282 -34.59599, -58.45366 -34.5...",27761.0,5369,112,193.400814,4.034437,...,251.0,296.0,431.0,4.0,322.0,25.0,821.0,101.0,240.0,260.0
1,PATERNAL,15,7087.513295,2.229829e+06,"POLYGON ((-58.46558 -34.59656, -58.46562 -34.5...",19717.0,3489,87,176.953898,4.412436,...,167.0,253.0,307.0,2.0,205.0,53.0,534.0,79.0,126.0,161.0
2,VILLA CRESPO,15,8131.857075,3.615978e+06,"POLYGON ((-58.42375 -34.59783, -58.42495 -34.5...",81959.0,13469,286,164.338267,3.489550,...,698.0,297.0,1156.0,8.0,893.0,79.0,2224.0,307.0,599.0,845.0
3,VILLA DEL PARQUE,11,7705.389797,3.399596e+06,"POLYGON ((-58.49461 -34.61487, -58.49479 -34.6...",55273.0,7692,176,139.163787,3.184195,...,454.0,136.0,731.0,10.0,584.0,25.0,1168.0,175.0,303.0,426.0
4,ALMAGRO,5,8537.901368,4.050752e+06,"POLYGON ((-58.41287 -34.61412, -58.41282 -34.6...",131699.0,21480,498,163.099188,3.781350,...,1142.0,716.0,1774.0,17.0,1364.0,179.0,3261.0,548.0,987.0,1456.0
